In [1]:
PIANO_GENERATOR_MODEL_PATH = 'piano_model'
PIANO_GENERATOR_MODEL_WEIGHTS_PATH = './piano models/piano_model_variational_autoencoder_weights'

In [24]:
import os

import glob
import pickle
import numpy
import pandas
import numpy as np
from itertools import chain
from pydub import AudioSegment
from music21 import converter, instrument, note, chord, stream
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from tensorflow import keras
from keras import layers

In [3]:
def get_notes(path):
    """ Get all the notes and chords from the midi files in the 'path' directory """
    notes = []

    print(len(glob.glob(path + '/*.mid')))
    for file in glob.glob(path + '/*.mid'):
        midi = converter.parse(file)

#         print('parsing %s' % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [4]:
def get_notes(path):
    """ Get all the notes and chords from the midi files in the 'path' directory """
    notes = []

    print(len(glob.glob(path + '/*.mid')))
    for file in glob.glob(path + '/*.mid'):
        midi = converter.parse(file)

#         print('parsing %s' % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [125]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    return (network_input, network_output)
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [126]:
notes = get_notes('midi_songs')

93


In [127]:
n_vocab = len(set(list(chain.from_iterable(notes))))
network_input, network_output = prepare_sequences(notes, n_vocab)

In [129]:
def all_idx(idx, axis):
    grid = np.ogrid[tuple(map(slice, idx.shape))]
    grid.insert(axis, idx)
    return tuple(grid)

In [143]:
def onehot_initialization(a, ncols):
    out = np.zeros(a.shape + (ncols,), dtype=int)
    out[all_idx(a, axis=2)] = 1
    return out

In [144]:
ncols = max(max(network_input))+1

In [145]:
a = onehot_initialization(np.array(network_input), ncols)

In [146]:
a.shape

(1868, 32, 254)

In [128]:
network_input

[[66,
  45,
  150,
  45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225],
 [45,
  150,
  45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225,
  202],
 [150,
  45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225,
  202,
  171],
 [45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225,
  202,
  171,
  234],
 [125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135

In [119]:
network_output.shape

(1868, 254)

[[66,
  45,
  150,
  45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225],
 [45,
  150,
  45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225,
  202],
 [150,
  45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225,
  202,
  171],
 [45,
  125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135,
  168,
  93,
  168,
  225,
  202,
  171,
  234],
 [125,
  45,
  97,
  220,
  97,
  45,
  14,
  33,
  45,
  125,
  45,
  144,
  221,
  109,
  49,
  235,
  230,
  221,
  203,
  144,
  221,
  16,
  49,
  135

## Model - Variational Autoencoder

### Sampling layer

In [53]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Encoder

In [59]:
x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(network_input)

In [60]:
x.shape

TensorShape([1868, 16, 32])

In [61]:
x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x.shape

TensorShape([1868, 512])

In [62]:
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

In [63]:
x.shape

TensorShape([1868, 16])

In [66]:
z_mean.shape, z.shape

(TensorShape([1868, 1]), TensorShape([1868, 1]))

In [67]:
x = layers.Dense(8 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((8, 64))(x)

In [68]:
x.shape

TensorShape([None, 8, 64])

In [69]:
x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv1DTranspose(1, 3, activation="sigmoid", padding="same")(x)

In [70]:
x.shape

TensorShape([None, 32, 32])

In [71]:
decoder_outputs.shape

TensorShape([None, 32, 1])

In [104]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(32, 1))
x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, 32, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 16, 32)       128         input_26[0][0]                   
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 8, 64)        6208        conv1d_17[0][0]                  
__________________________________________________________________________________________________
flatten_10 (Flatten)            (None, 512)          0           conv1d_18[0][0]                  
____________________________________________________________________________________________

### Decoder

In [105]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(8 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((8, 64))(x)
x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv1DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               1536      
_________________________________________________________________
reshape_14 (Reshape)         (None, 8, 64)             0         
_________________________________________________________________
conv1d_transpose_26 (Conv1DT (None, 16, 64)            12352     
_________________________________________________________________
conv1d_transpose_27 (Conv1DT (None, 32, 32)            6176      
_________________________________________________________________
conv1d_transpose_28 (Conv1DT (None, 32, 1)             97        
Total params: 20,161
Trainable params: 20,161
Non-trainable params: 0
_______________________________________________________

In [106]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction)
                    , axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [73]:
z_mean, z_log_var, z = encoder(network_input)

In [74]:
z_mean.shape, z_log_var.shape, z.shape

(TensorShape([1868, 1]), TensorShape([1868, 1]), TensorShape([1868, 1]))

In [76]:
reconstruction = decoder(z)

In [91]:
reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(network_input, reconstruction)
                    , axis=1
                )
            )

In [94]:
a = tf.reduce_sum(
                    keras.losses.binary_crossentropy(network_input, reconstruction)
                    , axis=1
                )
a.shape

TensorShape([1868])

In [92]:
reconstruction_loss

<tf.Tensor: shape=(), dtype=float32, numpy=39707.547>

In [95]:
kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
kl_loss.shape

TensorShape([1868, 1])

In [96]:
kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
kl_loss.shape

TensorShape([])

In [97]:
kl_loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.5830584>

In [110]:
network_output[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [109]:
network_input[0]

array([[ 3.3 ],
       [ 2.25],
       [ 7.5 ],
       [ 2.25],
       [ 6.25],
       [ 2.25],
       [ 4.85],
       [11.  ],
       [ 4.85],
       [ 2.25],
       [ 0.7 ],
       [ 1.65],
       [ 2.25],
       [ 6.25],
       [ 2.25],
       [ 7.2 ],
       [11.05],
       [ 5.45],
       [ 2.45],
       [11.75],
       [11.5 ],
       [11.05],
       [10.15],
       [ 7.2 ],
       [11.05],
       [ 0.8 ],
       [ 2.45],
       [ 6.75],
       [ 8.4 ],
       [ 4.65],
       [ 8.4 ],
       [11.25]])

In [107]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(network_input, epochs=30, batch_size=32)

Epoch 1/30
59/59 [==============================] - 1s 2ms/step - loss: -39592.5935 - reconstruction_loss: -336707.8750 - kl_loss: 140087.3906
Epoch 2/30
59/59 [==============================] - 0s 2ms/step - loss: -8155271.4750 - reconstruction_loss: -17687098.0000 - kl_loss: 6539811.5000
Epoch 3/30
59/59 [==============================] - 0s 2ms/step - loss: 398549030.8000 - reconstruction_loss: -289227488.0000 - kl_loss: 749905856.0000
Epoch 4/30
59/59 [==============================] - 0s 2ms/step - loss: -861245828.2667 - reconstruction_loss: -7298083840.0000 - kl_loss: 3560678912.0000
Epoch 5/30
59/59 [==============================] - 0s 2ms/step - loss: -25673014050.1333 - reconstruction_loss: -38773874688.0000 - kl_loss: 20879587328.0000
Epoch 6/30
59/59 [==============================] - 0s 2ms/step - loss: -39140297540.2667 - reconstruction_loss: -92101992448.0000 - kl_loss: 63865286656.0000
Epoch 7/30
59/59 [==============================] - 0s 2ms/step - loss: -54109333400